In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath("../../"))
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import tracing_v2_enabled
from src.config import Config
from src.rag import ChromaSlideStore
import pandas as pd

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "default"

**Table of contents**<a id='toc0_'></a>    
- [Измерение качества Retrieval](#toc1_)    
  - [Измерение качества через разметку](#toc1_1_)    
  - [RAGAS - оценка RAG без разметки](#toc1_2_)    
    - [Входные данные](#toc1_2_1_)    
    - [RAGAS для поиска презентаций](#toc1_2_2_)    
- [Загрузка датасета с google.sheets](#toc2_)    
- [Инициализация Retrieval](#toc3_)    
- [Правила для оценки (evaluators)](#toc4_)    
- [Запуск тестов](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Измерение качества Retrieval](#toc0_)
Для измерения качества я использую Langsmith.

## <a id='toc1_1_'></a>[Измерение качества через разметку](#toc0_)
В [таблице](https://docs.google.com/spreadsheets/d/1qWRF_o-RY1x-o-3z08iVb2akh0HS3ZNxVkZi6yoVsI4/edit?gid=0#gid=0) разменные презентации. Формат презентация-вопрос-слайды. Если в ответ выдали нужную презу, скор 1, иначе 0.

## <a id='toc1_2_'></a>[RAGAS - оценка RAG без разметки](#toc0_)
- [arxiv](http://arxiv.org/abs/2309.15217)
- [Ноутбук с туториалом](https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/evaluation_of_rag_with_ragas.ipynb)

RAGAS - метод оценки генерации ответов RAG-системами.

Идея: хотим оценивать ответы системы без разметки.

Решение: придумаем метрики, которые 'self-contained and referece-free'.

Что я не понял:
- Почему Answer Relevance считается через эмбеддинги? Можно же гптшку спросить..
- Зачем в Faithfulness разбиение на key-points? Опять же гптшка и без этого поймет


### <a id='toc1_2_1_'></a>[Входные данные](#toc0_)
В метриках RAGAS используются
- Question - Исходный запрос
- Answer - Ответ модели
- Contexts - Документы, которые выдал Retrieval

Метрики:
- Faithfulness - ответ основывается на найденном контексте.
- Answer Relevance - ответ соответствует вопросу.
- Context Relevance - ответ модели строго по делу, без нерелевантной информации.

### <a id='toc1_2_2_'></a>[RAGAS для поиска презентаций](#toc0_)
В этом проекте ответом на запрос является слайды из конкретной презентации. У нас есть Contexts, но нет Answer. Метрики Faithfulness и Сontext Relevance отпадают сразу.

Для метрики Answer Relevance можно попробовать Answer=`лучший слайд`. Идея метрики:
- Для каждого ответа RAG-системы попросим LLM сгенерировать $n$ вопросов $q_i$.
- Получим эмбеддинги этих вопросов $e_i$
- Вычислим Similarity с эмбеддингом исходного запроса $e$: $AR = Mean(Sim(e, e_i))$

Вот, что из этого вышло:
LLM Генерила примерно одинаковые вопросы. Они были на английском, и не похожи на исходный. Получились рандомные скоры.

Возможная причина: У них в примерах короткие вопросы и короткие ответы. LLM не знает что делать с большим описанием слайда. Примеры из их промпта:

```
--------EXAMPLES-----------
Example 1
Input: {
    "response": "Albert Einstein was born in Germany."
}
Output: {
    "question": "Where was Albert Einstein born?",
    "noncommittal": 0
}

Example 2
Input: {
    "response": "I don't know about the  groundbreaking feature of the smartphone invented in 2023 as am unaware of information beyond 2022. "
}
Output: {
    "question": "What was the groundbreaking feature of the smartphone invented in 2023?",
    "noncommittal": 1
}
-----------------------------
```


# <a id='toc2_'></a>[Загрузка датасета с google.sheets](#toc0_)

In [3]:
sheet_id = os.environ["BENCHMARK_SPREADSHEET_ID"]

csv_load_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
df = pd.read_csv(csv_load_url)
df.fillna(dict(page=""), inplace=True)
df.sample(5)

,pres_name,question,page,content,comment,MinScorer,Hyperbolic Scorer
11,4.Обзор уязвимостей и техник защиты для LLM_Ев...,Слайд с Трампом,15,general,Внезапно в такой постановке все фейлят,FAIL,FAIL
8,Kept_Подвижной состав РФ_2024 (20 стр),Про что рассказывал Сергей Казачков?,"2,20",text,NaN,PASS,PASS
20,4.Эволюция отбора кандидатов в системе товарны...,Презентация в которой был мем с Вовочкой (и та...,,visual,NaN,FAIL,FAIL
5,Kept_Подвижной состав РФ_2024 (20 стр),Презентация в которой показывали карту грузовы...,11,visual,NaN,NaN,NaN
1,ЯиП_Энергетический_переход_Вызовы_и_возможност...,Карта стоимости электроэнергии,24,visual,NaN,PASS,PASS


In [4]:
df["pres_name"].str.contains("pdf").sum()

0

# <a id='toc3_'></a>[Инициализация Retrieval](#toc0_)

In [5]:
config = Config()
llm = config.model_config.load_vsegpt(model="")

collection_name = "pres0"
embeddings = config.embedding_config.load_vsegpt()
storage = ChromaSlideStore("pres0", embedding_model=embeddings)


In [8]:
from src.rag.score import ExponentialScorer, HyperbolicScorer, MinScorer, ExponentialWeightedScorer, HyperbolicWeightedScorer
from src.rag import PresentationRetriever
from pprint import pprint

question = "Слайд с Трампом"
question = "Презентация с мемом про Трампа"

scorer = ExponentialScorer()
retriever = PresentationRetriever(storage=storage, scorer=scorer)

out = retriever.retrieve(question)
pprint(out)

{'contexts': [{'contexts': ['Slide 14:\n'
                            '\n'
                            '\n'
                            'Text Content:\n'
                            '\n'
                            'Заголовок: "Grok Илонa Маска без цензуры"\n'
                            '\n'
                            'Текст твита:\n'
                            'Max Zeff @ZeффMax\n'
                            '"Can you generate an image of Donald Trump '
                            'smoking a joint on the Joe Rogan show"\n'
                            '\n'
                            'Стилизация текста: заголовок выполнен крупным '
                            'шрифтом синего цвета, что привлекает внимание к '
                            'основной теме слайда. Текст твита представлен в '
                            'стандартном шрифте, что подчеркивает его '
                            'второстепенное значение.\n'
                            '\n'
                            '\n'
   

Убедимся, что vsegpt и openai дают одинаковые эмбеддинги

In [8]:
config.embedding_config.load_vsegpt()
oai_emb = config.embedding_config.load_openai()
vgpt_emb = config.embedding_config.load_vsegpt()
oai_emb.embed_query(question) == vgpt_emb.embed_query(question)

True

In [9]:
# from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import Faithfulness, LLMContextRecall
from ragas import SingleTurnSample, EvaluationDataset
from ragas.llms.base import LangchainLLMWrapper


llm = config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
llm = LangchainLLMWrapper(llm)

question = "When was the first super bowl?"
contexts = ["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]
answer = "The first superbowl was held on Jan 15, 1967"


sample = SingleTurnSample(
        user_input=question,
        response=answer,
        # reference="on January 15, 1967",
        retrieved_contexts=contexts
    )

dataset = EvaluationDataset(samples=[sample])

metrics = [Faithfulness(llm=llm)]
for m in metrics:
    m.__setattr__("llm", llm)
    m.__setattr__("embeddings", embeddings)

from ragas.integrations.langchain import EvaluatorChain
out = metrics[0].single_turn_score(sample)
print(out)

1.0


Через датасет

In [10]:
from ragas import evaluate
out = evaluate(dataset=dataset, llm=llm, metrics=metrics)
out

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'faithfulness': 1.0000}

# Загрузка датасета в Langsmith

In [ ]:
from langsmith import Client
from langsmith.utils import LangSmithError

# Create dataset
client = Client()
dataset_name = "RAGAS_5"

try:
    # check if dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("using existing dataset: ", dataset.name)
except LangSmithError:
    # if not create a new one with the generated query examples
    dataset = client.create_dataset(dataset_name=dataset_name)
    df_5 = df.sample(5)
    for i, row in df_5.iterrows():
        client.create_example(
            inputs=dict(question=row["question"]),
            outputs=dict(
                ground_truth="",
                pres_name=row["pres_name"],
                pages=[int(x) if x else -1 for x in row["page"].split(",")]
            ),
            dataset_id=dataset.id,
        )

    print("Created a new dataset: ", dataset.name)

using existing dataset:  RAGAS_5


# <a id='toc4_'></a>[Правила для оценки (evaluators)](#toc0_)

In [ ]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from ragas import SingleTurnSample, EvaluationDataset
from ragas.llms.base import LangchainLLMWrapper

@run_evaluator
def presentation_match(run, example) -> EvaluationResult:
    prediction = run.outputs["pres_info"]["pres_name"]
    match = int(prediction == example.outputs["pres_name"])
    return EvaluationResult(key="presentation_match", score=match)


def ragas_evaluator(metric):
    @run_evaluator
    async def evaluate(run, example) -> EvaluationResult:
        sample = SingleTurnSample(
            user_input=example.inputs["question"],
            response=run.outputs["answer"],
            # reference="",
            retrieved_contexts=run.outputs["contexts"],
        )
        score = await metric.single_turn_ascore(sample)

        return EvaluationResult(key=metric.name, score=score)
    return evaluate

## LLM as a judge

In [9]:
retriever = PresentationRetriever(storage=storage, scorer=scorer)
question = "Презентация с мемом про Трампа"
out = retriever.retrieve(question)

In [10]:
out

{'contexts': [{'pres_name': '4.Обзор уязвимостей и техник защиты для LLM_Евгений Кокуйкин_вер.3',
   'pages': [15, 16, 5, 8],
   'contexts': ['Slide 14:\n\n\nText Content:\n\nЗаголовок: "Grok Илонa Маска без цензуры"\n\nТекст твита:\nMax Zeff @ZeффMax\n"Can you generate an image of Donald Trump smoking a joint on the Joe Rogan show"\n\nСтилизация текста: заголовок выполнен крупным шрифтом синего цвета, что привлекает внимание к основной теме слайда. Текст твита представлен в стандартном шрифте, что подчеркивает его второстепенное значение.\n\n\nVisual Content:\n\nНа слайде изображение Дональда Трампа, который курит, находится справа. Он одет в темный костюм и наушники, что создает впечатление, что он участвует в разговоре. Слева расположен текст твита, который также содержит имя пользователя и его никнейм. Фон слайда черный, что делает текст и изображение более заметными.\n\n\nTopic Overview:\n\nТема: Обсуждение Илонa Маска и его влияния на общественное мнение.\nЦель: Показать провокац

In [ ]:
pres = out["contexts"][0]
pres_context = "\n\n---\n".join(pres["contexts"])
print(pres_context)

Slide 14:


Text Content:

Заголовок: "Grok Илонa Маска без цензуры"

Текст твита:
Max Zeff @ZeффMax
"Can you generate an image of Donald Trump smoking a joint on the Joe Rogan show"

Стилизация текста: заголовок выполнен крупным шрифтом синего цвета, что привлекает внимание к основной теме слайда. Текст твита представлен в стандартном шрифте, что подчеркивает его второстепенное значение.


Visual Content:

На слайде изображение Дональда Трампа, который курит, находится справа. Он одет в темный костюм и наушники, что создает впечатление, что он участвует в разговоре. Слева расположен текст твита, который также содержит имя пользователя и его никнейм. Фон слайда черный, что делает текст и изображение более заметными.


Topic Overview:

Тема: Обсуждение Илонa Маска и его влияния на общественное мнение.
Цель: Показать провокационный твит о Дональде Трампе.
Ключевая информация: Визуализация взаимодействия между известными личностями и их влиянием на медиа.


Conclusions And Insights:

Осно

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from textwrap import dedent

from pydantic import BaseModel, Field

class RelevanceOutput(BaseModel):
    explanation: str = Field(description="Explanation for the relevance score")
    relevance_score: int = Field(description="Relevance score (0 or 1)")

prompt_template = PromptTemplate.from_template(
"""\
You will act as an expert relevance assessor for a presentation retrieval system. Your task is to evaluate whether the retrieved slide descriptions contain relevant information for the user's query. Consider both textual content and references to visual elements (images, charts, graphs) as equally valid sources of information.

Evaluation Rules:
- Assign score 1 if the descriptions contain ANY relevant information that helps answer the query
- Assign score 0 only if the descriptions are completely unrelated or provide no useful information
- Treat references to visual elements (e.g., "graph shows increasing trend" or "image depicts workflow") as valid information
- Consider partial matches as relevant (score 1) as long as they provide some value in answering the query

For each evaluation, you will receive:
1. The user's query
2. Retrieved slide descriptions

# Query
{query}

--- END OF QUERY ---

# Slide Descriptions
{context}

--- END OF SLIDE DESCRIPTIONS ---

Format output as JSON:

```json
{{
  "explanation": string, # Clear justification explaining why the content is relevant or irrelevant
  "relevance_score": int  # 1 if any relevant information is found, 0 if completely irrelevant
}}
```
"""
)

parser = JsonOutputParser(pydantic_object=RelevanceOutput)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"explanation": {"description": "Explanation for the relevance score", "title": "Explanation", "type": "string"}, "relevance_score": {"description": "Relevance score (0 or 1)", "title": "Relevance Score", "type": "integer"}}, "required": ["explanation", "relevance_score"]}\n```'

In [43]:
retriever = PresentationRetriever(storage=storage, scorer=MinScorer())
question = "Презентация с мемом про Трампа"
question = "Презентация в которой показывали карту грузовых маршрутов для поездов"
question = "Презентация про маркетплейсы"
question = "Благополучие"
out = retriever.retrieve(question)
pres = out["contexts"][0]
pres_context = "\n\n---\n".join(pres["contexts"])

In [ ]:
from pprint import pprint

llm = config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
chain = prompt_template | llm | StrOutputParser() | parser
llm_out = chain.invoke(dict(query=question, context=pres_context))
pprint(llm_out)


{'explanation': 'The slide descriptions do not contain any relevant '
                'information directly related to the query about '
                "'Благополучие' (well-being). Slide 28 discusses problems and "
                'failures in the work process, emphasizing the importance of '
                'analyzing failures for growth, which is somewhat tangential '
                'to the concept of well-being. Slide 37 is a concluding slide '
                'that thanks the audience and provides a QR code for feedback, '
                'which is also unrelated to the topic of well-being. '
                'Therefore, there is no useful information to address the '
                "user's query.",
 'relevance_score': 0}


In [37]:
llm_out

{'explanation': "Slide 10 contains a map of major freight railway routes in Russia, which directly relates to the user's query about a presentation showing a map of freight routes for trains. The text discusses key freight basins and their export data, which adds relevant context to the visual element of the map. Therefore, this slide is highly relevant to the query.",
 'relevance_score': 1}

In [30]:
llm_out

{'explanation': "The retrieved slide descriptions do not contain a meme about Donald Trump, nor do they address the user's request for a presentation featuring such a meme. Slide 14 includes an image of Trump but focuses on a tweet about him rather than a meme. The other slides do not mention Trump or memes at all. Therefore, the content does not adequately answer the query.",
 'relevance_score': 0}

# <a id='toc5_'></a>[Запуск тестов](#toc0_)

In [ ]:
from langsmith import evaluate
from ragas.metrics import (
    AnswerCorrectness, AnswerRelevancy,
    ContextPrecision, ContextRecall,
    Faithfulness,
)
from ragas.llms.base import LangchainLLMWrapper

# Jupyter async hack
import nest_asyncio
nest_asyncio.apply()

# make eval chains
llm = config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
llm = LangchainLLMWrapper(llm)

# Setup metrics
eval_chains = {}
eval_chains["presentation_match"] = presentation_match

metrics = [Faithfulness]
for m in metrics:
    metric_with_llm = m(llm=llm)
    evaluator = ragas_evaluator(metric_with_llm)
    eval_chains.update({m.name: evaluator})

eval_chains_list = list(eval_chains.values())

# Setup scorers for RAG reranking
# scorers = [MinScorer(), HyperbolicScorer(), ExponentialScorer(), HyperbolicWeightedScorer(), ExponentialWeightedScorer()]
scorers = [MinScorer(), HyperbolicScorer()]

# Run evaluation. Results will be available in Langsmith
for scorer in scorers:
    retriever = Slideetriever(storage=storage, scorer=scorer)
    out = evaluate(
        retriever,
        experiment_prefix=f"{retriever.scorer.id}",
        data=dataset_name,
        evaluators=eval_chains_list,
        metadata=dict(
            scoreRr=retriever.scorer.id
        ),
        max_concurrency=2,
    )

**See the results in Langsmith**


---

**Deepeval (not finished)**

Эту часть я не доделал.

Deepeval - фреймворк для оценки моделей. Там есть метрики RAGAS. Можно оценивать через него

In [ ]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics.ragas import RAGASFaithfulnessMetric

from langchain_core.output_parsers import StrOutputParser
from deepeval.models.base_model import DeepEvalBaseLLM
import json

class LangchainModelEval(DeepEvalBaseLLM):
    def __init__(self, llm):
        self.llm = llm

    def load_model(self, *args, **kwargs):
        return self.llm

    def generate(self, prompt: str) -> str:
        chain = self.llm | StrOutputParser() # | json.loads
        out = chain.invoke(prompt)
        return out

    async def a_generate(self, prompt: str, *args, **kwargs) -> str:
        schema = kwargs.get("schema")
        out = self.generate(prompt)
        out = out if schema is None else schema(out)
        return out

    def get_model_name(self):
        llm_class = self.llm.__class__.__name__
        model_string = self.llm.model_name if hasattr(self.llm, "model_name") else self.llm.model
        model_string = model_string.replace("/", "-")
        return f"{llm_class}-{model_string}"


question = "When was the first super bowl?"
contexts = ["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]
answer = "The first superbowl was held on Jan 15, 1967"

llm = config.model_config.load_vsegpt(model="openai/gpt-4o-mini")
metrics = [RagasMetric(threshold=0.5, model=llm)]
metric = RAGASFaithfulnessMetric(model=llm)

test_case = LLMTestCase(
    input=question,
    actual_output=answer,
    retrieval_context=contexts
)

metric.measure(test_case)
print(metric.score)

In [19]:
list(client.list_projects())
list(client.list_datasets())

[Dataset(name='RAG_test_25', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('dc15e6f2-dea2-48f5-a9f1-76cbe7ebd172'), created_at=datetime.datetime(2024, 12, 1, 12, 25, 6, 335839, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 12, 1, 12, 25, 6, 335839, tzinfo=datetime.timezone.utc), example_count=25, session_count=8, last_session_start_time=datetime.datetime(2024, 12, 1, 14, 33, 43, 45139), inputs_schema=None, outputs_schema=None),
 Dataset(name='Pragmatics_12', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('9f113efd-afb3-4d0d-a21c-e64c30add6c3'), created_at=datetime.datetime(2024, 4, 4, 11, 10, 16, 777543, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 4, 4, 11, 10, 16, 777543, tzinfo=datetime.timezone.utc), example_count=11, session_count=1, last_session_start_time=datetime.datetime(2024, 4, 4, 11, 10, 38, 417340), inputs_schema=None, outputs_schema=None),
 Dataset(name='Chegeka_100', description=None, data_type=<DataType.kv: